In [3]:
from typing import List
import asyncio
from dataclasses import dataclass
from langchains.openai import run_llm_chain_for_input
from integrations.github import GithubFile


@dataclass
class ProcessedGithubFile:
    originalFile: GithubFile
    llmResponse: str

mock_github_results: List[GithubFile] = [
    GithubFile("test1.txt", "This is a test file"),
    GithubFile("test2.txt", "This is another test file"),
    GithubFile("test3.txt", "This file has a dog in it"),
    GithubFile("test4.txt", "This file has a cat in it"),
    GithubFile("test5.txt", "This file has a golden retriever in it"),
    GithubFile("test6.txt", "This is yet another test file"),
]
mock_user_prompt = "find dogs within the files"


# Define the function you want to run on each object:
async def process_item(item: GithubFile) -> ProcessedGithubFile:
    # This is a placeholder function that doesn't do much.
    # Replace it with whatever function you want to run.
    # await asyncio.sleep(1)  # simulate IO delay
    # return ProcessedGithubFile(item, f"Processed {item.filename} with code {item.content}")
    #return f"Processed {item.filename} with code {item.content}"

    result = run_llm_chain_for_input(mock_user_prompt, item.content)
    return ProcessedGithubFile(item, result)



# TODO since I think this syntax is dependent on the python version
# We should find a way to define it such that heroku and local development will respect/enforce it
async def process_all_items(items: List[GithubFile]) -> List[ProcessedGithubFile]:
    # Create a list of tasks to run:
    tasks = [process_item(item) for item in items]

    # Run the tasks:
    #loop = asyncio.get_event_loop()
    # results = loop.run_until_complete(asyncio.gather(*tasks))
    results = await asyncio.gather(*tasks)

    # Do something with the results:
    return results

In [4]:
results = await process_all_items(mock_github_results)
print(results)

[ProcessedGithubFile(originalFile=GithubFile(filename='test1.txt', content='This is a test file'), llmResponse='\n\n0'), ProcessedGithubFile(originalFile=GithubFile(filename='test2.txt', content='This is another test file'), llmResponse='\n\n50'), ProcessedGithubFile(originalFile=GithubFile(filename='test3.txt', content='This file has a dog in it'), llmResponse='\n\n100'), ProcessedGithubFile(originalFile=GithubFile(filename='test4.txt', content='This file has a cat in it'), llmResponse='\n\n0'), ProcessedGithubFile(originalFile=GithubFile(filename='test5.txt', content='This file has a golden retriever in it'), llmResponse='\n\n100'), ProcessedGithubFile(originalFile=GithubFile(filename='test6.txt', content='This is yet another test file'), llmResponse='\n\n50'), ProcessedGithubFile(originalFile=GithubFile(filename='test1.txt', content='This is a test file'), llmResponse='\n\n0'), ProcessedGithubFile(originalFile=GithubFile(filename='test2.txt', content='This is another test file'), ll

In [5]:
from integrations.github import download_github_data_to_file, get_github_files_and_contents

files = get_github_files_and_contents("https://github.com/chasemc67/TinyGen")
download_github_data_to_file(files)

In [6]:
from integrations.github import read_github_data_from_file

files = read_github_data_from_file()
print(files)

[GithubFile(filename='.gitignore', content='# Config secrets\n.env\n\n# Python cache files\n__pycache__/\n*.py[cod]\n\n# Virtual environment\nvenv/\n\n# Project-specific settings\n.env  # environment variables, never commit this\n*.log  # log files\n\n# Byte-compiled / optimized / DLL files\n*.py[cod]\n*$py.class\n\n# Distribution / packaging\n.Python\nbuild/\ndevelop-eggs/\ndist/\ndownloads/\neggs/\n.eggs/\nlib/\nlib64/\nparts/\nsdist/\nvar/\n*.egg-info/\n.installed.cfg\n*.egg\n\n# PyCharm files\n.idea/\n*.iml'), GithubFile(filename='Procfile', content='web: uvicorn main:app --host=0.0.0.0 --port=${PORT:-5000}'), GithubFile(filename='README.md', content='# TinyGen\n\nA simple API to generate code diffs using ChatGPT\n\n## Running the Application Locally\n\n1. First, clone the repository to your local machine and navigate to the project directory:\n\n```bash\ngit clone <repository-url>\ncd <project-directory>\nsource venv/bin/activate\npip install -r requirements.txt\n```\n\nTeardown:\

In [1]:
from app.generate_diff import find_relevant_files_for_prompt_and_repo

processedFiles = await find_relevant_files_for_prompt_and_repo("The file where the routes are defined. Should contain things like app.get", "https://github.com/chasemc67/TinyGen")
print(processedFiles)

[ProcessedGithubFile(originalFile=GithubFile(filename='.gitignore', content='# Config secrets\n.env\n\n# Python cache files\n__pycache__/\n*.py[cod]\n\n# Virtual environment\nvenv/\n\n# Project-specific settings\n.env  # environment variables, never commit this\n*.log  # log files\n\n# Byte-compiled / optimized / DLL files\n*.py[cod]\n*$py.class\n\n# Distribution / packaging\n.Python\nbuild/\ndevelop-eggs/\ndist/\ndownloads/\neggs/\n.eggs/\nlib/\nlib64/\nparts/\nsdist/\nvar/\n*.egg-info/\n.installed.cfg\n*.egg\n\n# PyCharm files\n.idea/\n*.iml'), llmResponse='\n    0'), ProcessedGithubFile(originalFile=GithubFile(filename='Procfile', content='web: uvicorn main:app --host=0.0.0.0 --port=${PORT:-5000}'), llmResponse='\n    0'), ProcessedGithubFile(originalFile=GithubFile(filename='README.md', content='# TinyGen\n\nA simple API to generate code diffs using ChatGPT\n\n## Running the Application Locally\n\n1. First, clone the repository to your local machine and navigate to the project dire

In [2]:
for file in processedFiles:
    print(file.originalFile.filename, file.llmResponse)
    print("\n")

.gitignore 
    0


Procfile 
    0


README.md 
    0


config.py 
    0


main.py 
    50


requirements.txt 
    0


runtime.txt  0


testbed.ipynb 
    0




In [1]:
from langchains.openai import run_test_llm_chain

result = run_test_llm_chain()
print(result)


    Answer: You have given the userPrompt as "what the user wants" and the codePrompt as "the users code".
